# 🧙 CellMage Multiple Folders Example ✨

This notebook demonstrates how to work with multiple persona and snippet folders in CellMage.

**Date:** April 27, 2025

## Why Use Multiple Folders?

Using multiple folders for personas and snippets provides several benefits:

1. **Organize by project**: Keep project-specific personas and snippets separate
2. **Share selectively**: Share only certain folders with collaborators
3. **Modular organization**: Group related personas and snippets together
4. **Environment-specific settings**: Different settings for different environments

## 1. Setting Up CellMage

First, let's load the CellMage extension and ensure it's properly initialized:

In [1]:
# Load the extension
%load_ext cellmage

# Import the necessary modules
import cellmage
import os
import logging

# Set up logging to see directory info
logging.basicConfig(level=logging.INFO)

print(f"CellMage version: {cellmage.__version__}")

✅ NotebookLLM Magics loaded. Use %llm_config and %%llm.
   For ambient mode, try %llm_config_persistent to process all cells as LLM prompts.
CellMage version: 0.1.0


## 2. Understanding Directory Structure

CellMage now automatically detects and uses multiple folders for personas and snippets. Let's see what directories are being used:

In [2]:
# Get the default manager
manager = cellmage.get_default_manager()

# Check what persona loader is being used
loader = manager.persona_loader

# If it's a MultiFileLoader, we can see the directories
if hasattr(loader, "personas_dirs") and hasattr(loader, "snippets_dirs"):
    print("Persona directories:")
    for dir_path in loader.personas_dirs:
        print(f" - {dir_path}")

    print("\nSnippet directories:")
    for dir_path in loader.snippets_dirs:
        print(f" - {dir_path}")
else:
    print("Not using MultiFileLoader. Current loader:", type(loader).__name__)

Persona directories:
 - llm_personas

Snippet directories:
 - llm_snippets
 - snippets


## 2.1 Enabling Auto-Save for Conversations

Let's check if the `llm_conversations` folder exists and enable automatic conversation saving:

In [ ]:
# Check if llm_conversations folder exists and enable auto-save
import os

# Define the path to the conversations folder
conversations_folder = "llm_conversations"

# Check if the folder exists
if os.path.exists(conversations_folder) and os.path.isdir(conversations_folder):
    print(f"Found '{conversations_folder}' folder. Enabling auto-save for conversations.")

    # Enable auto-save via the manager's settings
    manager.settings.auto_save = True
    print(f"Auto-save is now: {manager.settings.auto_save}")

    # You can also set the auto-save folder (optional)
    manager.settings.conversations_dir = conversations_folder
    print(f"Conversations will be saved to: {os.path.abspath(manager.settings.conversations_dir)}")

    # Display current auto-save settings
    print(f"Current autosave_file template: {manager.settings.autosave_file}")
else:
    print(f"Couldn't find '{conversations_folder}' folder. Auto-save is not enabled.")

## 3. Listing Available Resources

Let's see what personas and snippets are available across all directories:

In [3]:
# List available personas
personas = manager.list_personas()
print(f"Available personas ({len(personas)}):\n", "\n ".join(personas))

# List available snippets
snippets = manager.list_snippets()
print(f"\nAvailable snippets ({len(snippets)}):\n", "\n ".join(snippets))

Available personas (0):
 

Available snippets (0):
 


## 4. Creating Custom Directory Structure

Let's create a small project with its own persona and snippet folders:

In [4]:
# Create project directories
project_dir = "my_project"
personas_dir = os.path.join(project_dir, "personas")
snippets_dir = os.path.join(project_dir, "snippets")

os.makedirs(personas_dir, exist_ok=True)
os.makedirs(snippets_dir, exist_ok=True)

# Create a custom persona
with open(os.path.join(personas_dir, "project_expert.md"), "w") as f:
    f.write("""---
name: project_expert
temperature: 0.3
model: gpt-4.1-nano
---
You are a specialized assistant for 'my_project'. You have deep knowledge of this project's requirements and specifications. Always refer to the project documentation when answering questions.""")

# Create a custom snippet
with open(os.path.join(snippets_dir, "project_requirements.md"), "w") as f:
    f.write("""# Project Requirements

1. The system must be responsive under high load
2. All data must be encrypted at rest and in transit
3. The UI must be accessible according to WCAG 2.1 AA standards
4. Response time should not exceed 200ms for API calls""")

print(f"✅ Created project structure at {os.path.abspath(project_dir)}")

✅ Created project structure at /Users/tpinto/madpin/cellmage/notebooks/examples/my_project


## 5. Using Custom FileLoader

Now, let's create a custom loader to access our project-specific resources:

In [5]:
from cellmage.resources.file_loader import FileLoader, MultiFileLoader

# Create a loader for the project directory
project_loader = FileLoader(personas_dir=personas_dir, snippets_dir=snippets_dir)

# List available resources in the project
project_personas = project_loader.list_personas()
project_snippets = project_loader.list_snippets()

print(f"Project personas: {project_personas}")
print(f"Project snippets: {project_snippets}")

Project personas: ['project_expert']
Project snippets: ['project_requirements']


## 6. Using Environment Variables for Multiple Directories

You can also set environment variables to configure multiple directories:

In [6]:
# Set environment variables for multiple directories
import os

# Directory paths to use
standard_dir = "llm_personas"
project_dir_path = os.path.abspath(personas_dir)

# Set environment variable for multiple directories
os.environ["CELLMAGE_PERSONAS_DIRS"] = f"{standard_dir},{project_dir_path}"

# For snippet directories
os.environ["CELLMAGE_SNIPPETS_DIRS"] = f"llm_snippets,{os.path.abspath(snippets_dir)}"

# Now reload the settings
from cellmage.config import Settings

new_settings = Settings()

print("Configured persona directories:")
print(new_settings.all_personas_dirs)

print("\nConfigured snippet directories:")
print(new_settings.all_snippets_dirs)

Configured persona directories:
['llm_personas', '/Users/tpinto/madpin/cellmage/notebooks/examples/my_project/personas']

Configured snippet directories:
['llm_snippets', '/Users/tpinto/madpin/cellmage/notebooks/examples/my_project/snippets']


## 7. Creating a Combined Loader

We can create a combined loader that searches multiple directories:

In [7]:
# Create a multi-file loader with both the default and project directories
combined_loader = MultiFileLoader(
    personas_dirs=["llm_personas", personas_dir], snippets_dirs=["llm_snippets", snippets_dir]
)

# List all available resources
all_personas = combined_loader.list_personas()
all_snippets = combined_loader.list_snippets()

print(f"All personas: {all_personas}")
print(f"All snippets: {all_snippets}")

All personas: ['project_expert']
All snippets: ['project_requirements']


## 8. Using Personas and Snippets from Different Directories

Let's try using resources from different directories in a conversation:

In [8]:
# Set up LLM configuration with our project persona
%llm_config --persona project_expert --sys-snippet project_requirements

# Verify the configuration
%llm_config --show-persona

2025-04-27 13:58:01,498 - cellmage.resources.file_loader - WARNING - Snippet 'project_requirements.md' not found at llm_snippets/project_requirements.md
2025-04-27 13:58:01,499 - cellmage.chat_manager - WARNING - Snippet 'project_requirements' not found
2025-04-27 13:58:01,500 - cellmage.resources.file_loader - WARNING - Persona 'project_expert' not found in llm_personas


⚠️ Warning: Could not add system snippet 'project_requirements'.
❌ Error: Persona 'project_expert' not found.
Active Persona: None
  To set a persona, use: %llm_config --persona <name>
  To list available personas, use: %llm_config --list-personas


In [9]:
%%llm
What are the key performance requirements for this project?

To accurately identify the key performance requirements for your project, I would need more specific information about the project's scope, objectives, industry, and deliverables. However, generally speaking, key performance requirements typically include:

1. **Functional Requirements:** Specific features and functionalities that the project must deliver to meet user needs.

2. **Performance Metrics:** Quantitative standards such as response time, throughput, accuracy, or efficiency benchmarks.

3. **Reliability and Availability:** Uptime requirements, fault tolerance, and mean time between failures.

4. **Scalability:** Ability to handle increased load or user demand over time.

5. **Security:** Data protection, access controls, and compliance with relevant standards and regulations.

6. **Usability:** User experience standards, including ease of use and accessibility.

7. **Maintainability:** Ease of updates, troubleshooting, and long-term support.

8. **Integration:** Compatibility with existing systems, standards, or platforms.

9. **Compliance:** Adherence to legal, regulatory, or industry-specific standards.

If you can provide more details about the project’s type, industry, or specific goals, I can give a more tailored list of key performance requirements.

## 9. Advanced: Context Switching Between Projects

Let's create a utility to easily switch between different project contexts:

In [10]:
def switch_project_context(project_name):
    """Switch to a specific project context with its own personas and snippets."""
    if project_name == "default":
        # Use default settings
        %llm_config --persona coder --clear-history
        print(f"Switched to default project context")
    elif project_name == "my_project":
        # Use project-specific settings
        %llm_config --persona project_expert --sys-snippet project_requirements --clear-history
        print(f"Switched to 'my_project' context")
    else:
        print(f"Unknown project: {project_name}")


# Example usage
switch_project_context("my_project")

2025-04-27 13:58:02,777 - cellmage.resources.file_loader - WARNING - Snippet 'project_requirements.md' not found at llm_snippets/project_requirements.md
2025-04-27 13:58:02,777 - cellmage.chat_manager - WARNING - Snippet 'project_requirements' not found
2025-04-27 13:58:02,778 - cellmage.resources.file_loader - WARNING - Persona 'project_expert' not found in llm_personas


⚠️ Warning: Could not add system snippet 'project_requirements'.
❌ Error: Persona 'project_expert' not found.
✅ Chat history cleared.
Switched to 'my_project' context


In [11]:
%%llm
Based on the project requirements, what encryption standards would you recommend?

To recommend appropriate encryption standards based on your project requirements, I would need more details about the specific needs and context of your project. Could you please provide information such as:

- The type of data being protected (e.g., sensitive personal data, financial information, general data)
- Compliance or regulatory standards you need to meet (e.g., GDPR, HIPAA, PCI DSS)
- Performance considerations (e.g., encryption speed, latency)
- Environment (e.g., web application, mobile app, embedded systems, cloud)
- Whether you need data at rest, data in transit, or both
- Any existing infrastructure or technology constraints

With this information, I can give a more tailored recommendation. However, here are some commonly used encryption standards and best practices:

**1. Data at Rest:**
- **AES (Advanced Encryption Standard)**: Recommended for encrypting stored data. Use AES-256 for high security.
- **Full Disk Encryption (FDE)** solutions like BitLocker (Windows) or FileVault (macOS)

**2. Data in Transit:**
- **TLS (Transport Layer Security)**: Use the latest version (currently TLS 1.3) for secure communication channels.
- **SSL** is outdated and should be avoided; prefer TLS.

**3. Key Management:**
- Use hardware security modules (HSMs) or dedicated key management systems.
- Follow best practices for generating, storing, and rotating keys.

**4. Hashing and Integrity:**
- Use SHA-256 or stronger algorithms for hashing.
- Employ HMAC for message authentication.

**5. Additional Considerations:**
- For applications requiring quantum resistance in the future, consider post-quantum cryptography standards when available.
- Ensure proper implementation to avoid vulnerabilities related to poor key management or algorithm misuse.

Please provide more details, and I can assist you with a more specific recommendation tailored to your project.

In [12]:
# Switch back to default context
switch_project_context("default")

2025-04-27 13:58:05,568 - cellmage.resources.file_loader - WARNING - Persona 'coder' not found in llm_personas


❌ Error: Persona 'coder' not found.
✅ Chat history cleared.
Switched to default project context


## Conclusion

Using multiple directories for personas and snippets allows you to:

1. **Organize resources** by project or domain
2. **Switch contexts** easily when working on different projects
3. **Share selective resources** with collaborators
4. **Maintain separation** between different types of content

The `MultiFileLoader` automatically searches across all directories, making it seamless to use resources from any location without having to remember exactly where each file is stored.